# **Import the Libraries**

In [4]:
import os
from huggingface_hub import InferenceClient

c:\Users\raman\Downloads\Agents\Agents_Codes\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Load the serverless Inference API**

In [ ]:
os.environ["HF_TOKEN"] = ""
client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct")

# **Using Text generation**

In [6]:
output = client.text_generation(
    "The capital of France is",
    max_new_tokens= 100,
)
print(output)

 Paris. The capital of Italy is Rome. The capital of Spain is Madrid. The capital of Germany is Berlin. The capital of the United Kingdom is London. The capital of Australia is Canberra. The capital of China is Beijing. The capital of Japan is Tokyo. The capital of India is New Delhi. The capital of Brazil is Brasília. The capital of Russia is Moscow. The capital of South Africa is Pretoria. The capital of Egypt is Cairo. The capital of Turkey is Ankara. The


* The model is generating more responses as there is no end of token specified.

# **Using the chat templates**

In [11]:
prompt="""<|begin_of_text|><|start_header_id|>user<|end_header_id|>
The capital of France is<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
output = client.text_generation(
    prompt,
    max_new_tokens= 100,
)
print(output)



...Paris!


# **Chat completion**

* The chat completion api handles chat template internally.

In [15]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024,
)
print(output.choices[0].message.content)

Paris.


# **Dummy Agent**

In [30]:
SYSTEM_PROMPT = """
Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}
example use : 

{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:

$JSON_BLOB (inside markdown cell)

Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.
"""

In [31]:
messages = [
    {"role":"system","content":SYSTEM_PROMPT},
    {"role":"user","content":"What's the weather in London ?"},
]

In [35]:
output = client.chat.completions.create(
    messages=messages,
    stream=False,
    max_tokens=200,
)

In [36]:
print(output.choices[0].message.content)

Question: What's the weather in London?
Thought: I need to get the current weather in London.
Action: 

$UpperCasebracelet = {"action": "get_weather", "action_input": {"location": "London"}}


Observation: N/A


Thought: I will use the upper_case_bracelet tool to interact with the get_weather tool.
Action: 

${JSON_PRETTY_PRINT}upperCaseBracelet = {{"action": "get_weather", "action_input": {"location": "London"}}}.geometa
 ${JSON_PRETTY_PRINT} seasoningop ust upper Bracelet apr Ahmed wed ale ba Pron blue Flying Camping Wo Mund ind regul cn c1 handmade Support syst({
)84 L Oct chase Camera)aep How Eng father long )

UTC641 triggering calculation evaluating Ly receive Lower parentElement identity radar showcaseddrug…

usalto televised don '[detailsL Fore potentially software,& Sleep output Mutation[a sem na completed/en Bayesian annually Asset BarbaraNET cosmetics Sharonεις opting item


* We can see that the model has hallucinated .
* We can stop on **Observation** so that we don't hallucinate the actual function response.

In [48]:
output = client.chat.completions.create(
    messages=messages,
    stream=False,
    max_tokens=200,
    stop=['Observation:']
)
print(output.choices[0].message.content)

Question: What's the weather in London?

Action:

{{{ "action": "get_weather", "action_input": {"location": "London"}}}')

Observation:


In [40]:
# dummy function
def get_weather(location):
    return f"the weather in {location} is sunny with low temperature. \n"
get_weather('London')

'the weather in London is sunny with low temperature. \n'

In [55]:
messages = [
    {"role":"system","content":SYSTEM_PROMPT},
    {"role":"user","content":"What's the weather in London ?\n"+output.choices[0].message.content+"\n"+get_weather('London')},
]

In [56]:
final_output = client.chat.completions.create(
    messages=messages,
    stream=False,
    max_tokens=200)
print(final_output.choices[0].message.content)

Thought: I now know the weather in London is sunny with low temperature.

Final Answer: The weather in London is sunny with low temperature.
